# Problem Statement

The problem statement is to predict the Surge Price Type 
 This is a multi-Label classification problem statement.

<img src ="img/2.png">

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample = pd.read_csv("data/sample_submission.csv")

In [5]:
train.shape,test.shape

((131662, 14), (87395, 13))

In [3]:
train.head()

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2


In [4]:
test.head()

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender
0,T0005689459,9.44,A,10.0,2.57438,A,B,3.68000,2,NaN,46,63,Male
1,T0005689462,32.15,B,10.0,2.85143,A,A,1.59375,0,65.0,49,80,Female
2,T0005689463,10.38,C,4.0,2.70530,A,D,4.50500,0,NaN,47,74,Male
3,T0005689466,14.94,NaN,6.0,2.48159,C,E,4.53000,0,63.0,43,54,Male
4,T0005689468,32.03,B,7.0,2.81598,A,K,4.60125,3,96.0,44,56,Male


In [6]:
train.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    20210
Customer_Since_Months           5920
Life_Style_Index               20193
Confidence_Life_Style_Index    20193
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                           71030
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type                 0
dtype: int64

In [7]:
train['Surge_Pricing_Type'].value_counts()

2    56728
3    47720
1    27214
Name: Surge_Pricing_Type, dtype: int64

## Modelling

In [8]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import log_loss,accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier,\
                                BaggingClassifier,ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.svm import SVC

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

## Cross Validation

In [9]:
def cross_valid(model,X,y,cv=3):
    results = cross_val_score(model, X,y,cv=cv,scoring='accuracy')
    return results.mean()

In [10]:
features = list(set(train.columns)-set(['Trip_ID', 'Surge_Pricing_Type']))
target = 'Surge_Pricing_Type'
features

['Cancellation_Last_1Month',
 'Trip_Distance',
 'Var2',
 'Var3',
 'Life_Style_Index',
 'Confidence_Life_Style_Index',
 'Customer_Rating',
 'Customer_Since_Months',
 'Type_of_Cab',
 'Destination_Type',
 'Gender',
 'Var1']

In [15]:
X_train = train.copy()
X_test = test.copy()

In [16]:
for i in ['Type_of_Cab','Confidence_Life_Style_Index']:
    X_train[i] = X_train[i].astype(str)
    X_test[i] = X_test[i].astype(str)

## Train & test split

In [17]:
trainX, validX, trainY, validY = train_test_split(X_train[features], 
                                                  X_train[target], test_size=0.2, random_state=13)

In [29]:
model= cat.CatBoostClassifier(iterations=1000,
                              verbose=0,
                              random_state=10000,
                              early_stopping_rounds=50,
                              cat_features=['Type_of_Cab','Confidence_Life_Style_Index','Destination_Type','Gender']
    )

In [32]:
from sklearn.multiclass import OneVsRestClassifier
model = OneVsRestClassifier(model)

model.fit(X=trainX,
         y=trainY)

OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7fb1ac050750>,
                                                                                n_jobs=None),
                                                  n_jobs=None),
                    n_jobs=None)

In [33]:
y_valid = model.predict(validX)
accuracy_score(validY,y_valid)

0.699692401169635

In [34]:
y_test = model.predict(X_test[features])
test['Surge_Pricing_Type']  = y_test
test[['Trip_ID','Surge_Pricing_Type']].to_csv("sampleSubmission.csv",index=False)

**Note: This model gave 0.7054 public score but 0.7064 private score**